## Function for converting images to nyt vision

In [ ]:

import cv2
import numpy as np
from PIL import Image, ImageEnhance
from numpy import asarray
import os
import pandas as pd
import os

def vignette_new(image, level = 2):
    height, width = image.shape[:2]

    x_resultant_kernel = cv2.getGaussianKernel(width, width/level)
    y_resultant_kernel = cv2.getGaussianKernel(height, height/level)

    kernel = y_resultant_kernel * x_resultant_kernel.T
    mask = kernel / kernel.max()

    image_vignette = np.copy(image)

    for i in range(3):
        image_vignette[:,:,i] = image_vignette[:,:,i] * mask

    return image_vignette


def convert_to_night_vision_new(imge,grayscale_contrast=0.7,brightness = 0.65,
                            amp_rc = 1.65,amp_gc = 1.0,amp_bc = 1.7):
    # Open the image
    # image = Image.open(image_path)
    imgg = Image.fromarray(imge)

    # Convert the image to grayscale
    grayscale_image = imgg.convert("L")

    # Enhance the contrast
    enhanced_image = ImageEnhance.Contrast(grayscale_image).enhance(grayscale_contrast)

    # Convert the image to RGB mode
    rgb_image = enhanced_image.convert("RGB")

    # Lowering Brightness of image to check low light
    rgb_image = ImageEnhance.Brightness(rgb_image).enhance(brightness)

    # Split the image into individual color channels
    r, g, b = rgb_image.split()

    # Amplify the green color channel
    amplified_r_channel = ImageEnhance.Brightness(r).enhance(amp_rc)
    amplified_g_channel = ImageEnhance.Brightness(g).enhance(amp_gc)
    amplified_b_channel = ImageEnhance.Brightness(b).enhance(amp_bc)

    # Create a new RGB image using amplified green channel
    night_vision_image = Image.merge("RGB", (amplified_r_channel, amplified_g_channel, amplified_b_channel))

    #VIGNETTE
    input = asarray(night_vision_image)

    # input
    vig_img = vignette_new(input,3)

    # Save the night vision image
    # image2 = Image.fromarray(vig_img)
    new_img = np.asarray(vig_img)

    return vig_img


In [ ]:
# define the path
path = "D:/Cat_id/yolo_eyes_nose_exp/data/classification_crop/from_effint/front_facing1/"

for files in os.listdir(path):
    file = files.split(".")[0]
    imge = cv2.imread(path + files )
    nyt_img =   convert_to_night_vision_new(imge,grayscale_contrast=0.7,brightness = 0.65,amp_rc = 1.65,amp_gc = 1.0,amp_bc = 1.7)
    cv2.imwrite(files , nyt_img)